In [1]:
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
import joblib

from collections import Counter
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
height, width = 10, 10

In [2]:
data = pd.read_csv(f"../archive/ascii_character_classification_{height}_x_{width}.csv", header=0).sample(frac=.05)

label_counts = Counter(data.iloc[:, 0])
print(label_counts)

Counter({0: 2542, 4: 274, 90: 267, 62: 266, 33: 265, 49: 263, 64: 257, 29: 257, 59: 255, 72: 255, 77: 252, 79: 252, 9: 251, 17: 251, 56: 250, 66: 250, 91: 250, 18: 249, 11: 248, 81: 248, 44: 247, 71: 247, 75: 247, 38: 247, 51: 247, 45: 246, 35: 246, 3: 245, 28: 244, 25: 242, 26: 242, 20: 242, 40: 241, 63: 241, 41: 240, 76: 240, 61: 240, 73: 239, 1: 238, 86: 238, 23: 238, 8: 238, 78: 238, 89: 238, 31: 237, 83: 237, 5: 237, 65: 236, 15: 236, 57: 236, 74: 236, 22: 236, 54: 235, 24: 235, 43: 235, 42: 234, 95: 234, 67: 233, 58: 233, 92: 233, 12: 232, 30: 230, 80: 230, 34: 230, 21: 230, 10: 229, 46: 229, 37: 229, 52: 228, 19: 228, 48: 228, 69: 228, 82: 228, 36: 226, 50: 226, 85: 226, 60: 226, 53: 225, 27: 225, 68: 225, 32: 224, 84: 223, 70: 223, 13: 223, 94: 223, 2: 222, 7: 221, 16: 221, 14: 219, 39: 217, 6: 214, 93: 211, 47: 210, 55: 209, 88: 204, 87: 202})


In [3]:
X = data.iloc[:, 1:].astype("float64")   # Features are all columns except the first one
y = data.iloc[:, 0].astype("float64")     # Labels are the first column
# Initialize the oversampler


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
# Optionally, if you want to convert them back to pandas DataFrames:
train_data = pd.concat([y_train, X_train], axis=1)
test_data = pd.concat([y_test, X_test], axis=1)

In [4]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
joblib.dump(clf, '../artifacts/svm_model.pkl')

['../artifacts/svm_model.pkl']

In [5]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (5000, 100)
Train Accuracy: 94.8000%
Test Accuracy: 93.8200%
F1 Score: 92.9271%
Recall: 93.8200%


In [6]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        image_reshaped = image.reshape((height, width))
        features = hog(image_reshaped, pixels_per_cell=(2, 2), cells_per_block=(1, 1), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

X_hog = extract_hog_features(np.array(X))

X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)


In [7]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
joblib.dump(clf, '../artifacts/svm_model_hog_10_x_10.pkl')

['../artifacts/svm_model_hog_10_x_10.pkl']

In [8]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (5000, 225)
Train Accuracy: 96.9200%
Test Accuracy: 94.4600%
F1 Score: 93.5780%
Recall: 94.4600%


In [9]:
import cv2

def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    
    for image in images:
        image_reshaped = image.reshape((height, width)).astype(np.uint8)
        keypoints, descriptors = sift.detectAndCompute(image_reshaped, None)
        
        # If no keypoints are found, use a zero array of the same length as a typical descriptor
        if descriptors is None:
            descriptors = np.zeros((1, sift.descriptorSize()), dtype=np.float32)
        
        # Flatten descriptors and use them as features
        features = descriptors.flatten()
        sift_features.append(features)
    
    return np.array(sift_features)

# Extract SIFT features
X_sift = extract_sift_features(np.array(X))

In [10]:
X_sift = extract_sift_features(np.array(X))

# Since the number of features might vary, we need to ensure consistent feature vector size
# Here, we'll pad with zeros to the maximum descriptor length found
max_len = max(len(f) for f in X_sift)
X_sift = np.array([np.pad(f, (0, max_len - len(f)), 'constant') for f in X_sift])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sift, y, test_size=0.2, random_state=42)

In [11]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
joblib.dump(clf, '../artifacts/svm_model_sift_10_x_10.pkl')

['../artifacts/svm_model_sift_10_x_10.pkl']

In [12]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (5000, 128)
Train Accuracy: 10.0300%
Test Accuracy: 10.7200%
F1 Score: 2.0758%
Recall: 10.7200%
